## Aim of this class is to get familiar with estimating ``activity'' of galaxies, cont.

Some new questions to answer:
- what are the drawbacks of BPT diagram?
- what if we don't have reliable measurements of some of the lines?

do **NOT** copy the catalogues (they'll eat up your quota), use them directly via:

In [ ]:
szuflada_path = '/home/SZUFLADA/PN/extragalactic_classes/gama_dr4/'
szuflada_path = './'

lines_file = szuflada_path + 'GaussFitComplexv05.fits'
lines_file = szuflada_path + '../agn_vs_dust/gama_data_dr4/GaussFitComplexv05.fits'

some imports:

In [ ]:
import pandas as pd
pd.options.mode.copy_on_write = True

from astropy.io import fits
from astropy.table import Table

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import astropy.units as u

### TASK: explore the ``equivalent width of Hα (EWHα) vs. [N ii]/Hα'' (WHAN) diagram, based on GAMA galaxies

TODOs:
- create a WHAN diagram for GAMA galaxies
- mark on the diagram different classes of galaxies
- mark division lines between the classes

some usefull pointers:
- Cid Fernandes+2010,**2011**
- SpecLineSFR DMU from GAMA DR4
- some other papers about WHAN diagram for GAMA galaxies ;)

### TASK: compare the BPT and WHAN diagrams, what are the differences? which one is ``better''?
write down your answer